# Load parish maps data into an SQLite database

In [34]:
import pandas as pd
from geojson import Feature, FeatureCollection, MultiLineString
import shapely
import geojson
import json
from pathlib import Path
from sqlite_utils import Database
import pandas as pd
from ast import literal_eval
from requests_cache import CachedSession
from tqdm.auto import tqdm
import time
import re

sess = CachedSession()
tqdm.pandas()

In [35]:
# Load parish maps data
df = pd.read_csv("parish_maps_final.csv")
# Load georeferenced maps data
dfg = pd.read_csv("https://raw.githubusercontent.com/wragge/slv-allmaps/refs/heads/main/georeferenced_maps.csv")

In [36]:
# Convert bboxes to lists
df["bbox"] = df.loc[df["bbox"].notnull()]["bbox"].apply(literal_eval)

In [37]:
df.head()

,alma_id,title,coordinates,east,south,publisher,description,image_id,latitude,longitude,height,width,scale,ratio,bbox,iiif_ids
0,9921177713607636,"Paaratte, County of Heytesbury",(E 142°59'/S 38°33').,E1425900,S0383300,"Melbourne :Dept. of Lands and Survey,1882.",1 map ;on sheet 58 x 42 cm.,IE7030338,-38.550000,142.983333,58.0,42.0,"Scale [ca. 1:31,680]",31680.0,"[142.90692800485053, -38.63273691139862, 143.0...",https://rosetta.slv.vic.gov.au:2083/iiif/2/IE7...
1,9921177763607636,"Powlett, County of Gladstone",(E 143°51'/S 36°25').,E1435100,S0362500,"Melbourne :Dept. of Lands and Survey,1883.",1 map ;on sheet 33 x 69 cm.,IE7041050,-36.416667,143.850000,33.0,69.0,"Scale [ca. 1:31,680]",31680.0,"[143.72806259777028, -36.46371024342854, 143.9...",https://rosetta.slv.vic.gov.au:2083/iiif/2/IE7...
2,9921177843607636,"Patho, County of Gunbower",(E 144°22'/S 35°59').,E1442200,S0353900,"Melbourne :H. J. Green, Govt. Printer,1926.",1 map ;on sheet 54 x 81 cm.,IE7022188,-35.650000,144.366667,54.0,81.0,Scale [ca. 1:31 680],31680.0,"[144.2248526147727, -35.72700847677786, 144.50...",https://rosetta.slv.vic.gov.au:2083/iiif/2/IE7...
3,9921177883607636,[Map of Parish of Stewarton],(E 145°50'/S 36°25' ).,E1455000,S0362500,"[Melbourne? :Dept. of Lands and Survey?,188-?].",1 map ;on sheet 30 x 41 cm.,IE7029775,-36.416667,145.833333,30.0,41.0,Scale not given,NaN,NaN,https://rosetta.slv.vic.gov.au:2083/iiif/2/IE7...
4,9921178363607636,Plan of a road from Stanley to Hillsborough an...,NaN,NaN,NaN,"Melbourne :Dept. of Lands and Survey,1867.",1 map ;on sheet 56 x 38 cm.,IE7041003,NaN,NaN,56.0,38.0,"Scale [ca. 1:15,840]",15840.0,NaN,https://rosetta.slv.vic.gov.au:2083/iiif/2/IE7...


In [38]:
dfg.head()

,allmaps_map_id,allmaps_manifest_id,image_id,ie_id,fl_id,map_title,manifest_title,area,modified,created,geometry
0,https://annotations.allmaps.org/maps/da6c67975...,https://annotations.allmaps.org/manifests/b855...,https://rosetta.slv.vic.gov.au/iiif/2/IE738834...,IE7388348,FL21479489,Map 1,Korong Vale,1.695382e+08,2025-11-26T09:59:52.487Z,2025-11-26T09:59:52.487Z,"{'type': 'Polygon', 'coordinates': [[[143.6242..."
1,https://annotations.allmaps.org/maps/19d1f0f6c...,https://annotations.allmaps.org/manifests/57fa...,https://rosetta.slv.vic.gov.au/iiif/2/IE738770...,IE7387701,FL21479634,Map 1,Wedderburn,1.637490e+08,2025-11-26T09:56:08.766Z,2025-11-26T09:56:08.766Z,"{'type': 'Polygon', 'coordinates': [[[143.4977..."
2,https://annotations.allmaps.org/maps/f203ed8ab...,https://annotations.allmaps.org/manifests/0285...,https://rosetta.slv.vic.gov.au/iiif/2/IE738800...,IE7388006,FL21479449,Map 1,Fiery Flat,1.605166e+08,2025-11-26T09:52:29.769Z,2025-11-26T09:52:29.769Z,"{'type': 'Polygon', 'coordinates': [[[143.7496..."
3,https://annotations.allmaps.org/maps/9de86c2de...,https://annotations.allmaps.org/manifests/a7d1...,https://rosetta.slv.vic.gov.au/iiif/2/IE738962...,IE7389623,FL21479407,Map 1,Serpentine,1.627654e+08,2025-11-26T09:49:06.636Z,2025-11-26T09:49:06.636Z,"{'type': 'Polygon', 'coordinates': [[[143.8734..."
4,https://annotations.allmaps.org/maps/9fdac7b7e...,https://annotations.allmaps.org/manifests/eb7d...,https://rosetta.slv.vic.gov.au/iiif/2/IE739157...,IE7391573,FL21479319,Map 1,Bears Lagoon,1.632794e+08,2025-11-26T09:45:48.835Z,2025-11-26T09:45:48.835Z,"{'type': 'Polygon', 'coordinates': [[[143.8728..."


In [39]:
geo_ids = dfg["ie_id"].to_list()

In [40]:
# add a boolean field to parish maps to indicate if it's been georeferenced
df["georeferenced"] = df["image_id"].apply(lambda x: 1 if x and x in geo_ids else 0)

In [41]:
def make_wkt(bbox):
    try:
        wkt = shapely.geometry.box(bbox[0],bbox[1],bbox[2],bbox[3]).wkt
    except IndexError:
        print(bbox)
        wkt = ""
    return wkt

# Create a wkt representation of the bounding boxes
df["wkt"] = df.loc[df["bbox"].notnull()]["bbox"].apply(make_wkt)

[]
[]


In [42]:
db = Database("spatial-datasette/parish_maps.db", recreate=True)

In [43]:
# Load all the data
db["parish_maps"].insert_all(df.to_dict(orient="records"))

<Table parish_maps (alma_id, title, coordinates, east, south, publisher, description, image_id, latitude, longitude, height, width, scale, ratio, bbox, iiif_ids, georeferenced, wkt)>

In [44]:
import sqlite3

conn = sqlite3.connect("spatial-datasette/parish_maps.db")
# Lead the spatialite extension:
conn.enable_load_extension(True)
conn.load_extension("/usr/lib/x86_64-linux-gnu/mod_spatialite.so")
# Initialize spatial metadata for this database:
conn.execute("select InitSpatialMetadata(1)")
# Add a geometry column called point_geom to our museums table:
conn.execute(
    "SELECT AddGeometryColumn('parish_maps', 'geometry', 4326, 'POLYGON', 'XY', 0);"
)
# Now update that geometry column with the lat/lon points
# I have had any success in trying to use the GeoJson column directly with GeomFromGeoJSON, 
# so instead I'm using a wkt column and GeoFromText
conn.execute(
    "UPDATE parish_maps SET geometry = GeomFromText(wkt, 4326) WHERE wkt is not null;"
)
# Now add a spatial index to that column
conn.execute(
    'select CreateSpatialIndex("parish_maps", "geometry");'
)
# If you don't commit your changes will not be persisted:
conn.commit()
conn.close()

db["parish_maps"].transform(drop={"wkt"})

<Table parish_maps (alma_id, title, coordinates, east, south, publisher, description, image_id, latitude, longitude, height, width, scale, ratio, bbox, iiif_ids, georeferenced, geometry)>